In [ ]:
import pandas as pd
import schedule
import datetime
import requests
import re
import time
import json
import lark_oapi as lark
from lark_oapi.api.bitable.v1 import *
from lark_oapi.api.im.v1 import *

def get_access_token():
    params = {
        'app_id':'你的飞书开发者后台机器人ID',
        'app_secret':'你的飞书开发者后台机器人密码'
    }
    response = requests.post("https://open.feishu.cn/open-apis/auth/v3/tenant_access_token/internal/", params = params)
    if response.status_code == 200:
        access_token_data = response.json()
        access_token = access_token_data.get('tenant_access_token')
        print("Successed to get access token.")
        return access_token
    else:
        print(f"Error:{response.text}")
        return None

def is_not_left(df):
    result = []
    for i in range(len(df)):
        j = df.loc[i,'管理人员确认离职时间']
        if type(j) == int:
            df.loc[i,'管理人员确认离职时间'] = datetime.datetime.fromtimestamp(j).strftime('%Y/%m/%d %H:%M:%S')
            j = datetime.datetime.fromtimestamp(j).strftime('%Y/%m/%d %H:%M:%S')
        if type(j) == datetime.datetime:
            result.append(False)
        elif type(j) == str:
            if j == '离职' or j == '已离职':
                result.append(False)
            elif re.match(pattern='.*\d.*\d.*', string=j):
                result.append(False)
            else:
                result.append(True)
        elif pd.isnull(j):
            result.append(True)
        else:
            print('离职时间格式存在错误，错误位于第'+str(int(i+1))+'行，内容为'+str(j)+'，格式为'+str(type(j))+'，所记录人员名称为'+str(df.loc[i,'请输入姓名']))
            result.append(False)
            continue
    return df[result]

def is_left_error(df):
    errors = []
    for i in range(len(df)):
        if pd.isnull(df.loc[i,'管理人员确认离职时间']):
            if re.search('离职',str(df.loc[i,'合同签署情况'])):
                errors.append(i)
    if errors:
        for err in errors:
            print('离职状态异常警告，错误位于第'+str(int(err+1))+'行，离职时间为'+str(df.loc[err,'管理人员确认离职时间'])+'，合同签署情况为'+str(df.loc[err,'合同签署情况'])+
                  '，所记录人员名称为'+str(df.loc[err,'请输入姓名']))
    return errors

def is_new(data_df,df):
    result = []
    for i in range(len(data_df)):
        if data_df.loc[i,'key'] in list(set(df['key'])):
            result.append(False)
        elif data_df.loc[i,'姓名'] in list(set(df['请输入姓名'])):
            result.append(False)
        else:
            result.append(True)
    return result

def at_someone(lis,name2openid_dict):
    final_text = ''
    text = ''
    for name in lis.split('/'):
        if name in name2openid_dict:
            openid = name2openid_dict[name]
            text = f"<at user_id=\\\"{openid}\\\">{name}</at>"
        else:
            text = f"@{name}"
        final_text = final_text + text
    return final_text

def job():
    open_id_dict = {
        '管理人员姓名1': '企业内飞书用户ID，可调用飞书API获取'，
        '管理人员姓名2': '企业内飞书用户ID，可调用飞书API获取'
        #......
    }

    location_dict = {
        '管理人员姓名1': '管理人员所在部门1',
        '管理人员姓名2': '管理人员所在部门2',
        #......
        '？':'未知部门'
        }

    # 创建client
    # 使用 user_access_token 需开启 token 配置, 并在 request_option 中配置 token
    client = lark.Client.builder() \
        .enable_set_token(True) \
        .log_level(lark.LogLevel.DEBUG) \
        .build()

    all_records = []
    page_token = ''

    while True:
        # 构造请求对象
        request: SearchAppTableRecordRequest = SearchAppTableRecordRequest.builder() \
            .app_token("你的表格app token") \
            .table_id("多维表格ID") \
            .page_size(500) \
            .page_token(page_token) \
            .request_body(SearchAppTableRecordRequestBody.builder()
                .view_id("你的view id")
                .field_names(["姓名", "公司名称", "日期", "电话号码","直属上级"])
                .filter(FilterInfo.builder()
                    .conjunction("and")
                    .conditions([Condition.builder()
                        .field_name("日期")
                        .operator("is")
                        .value(["Today"])
                        .build()
                        ])
                    .build())
                .automatic_fields(False)
                .build()) \
            .build()

        # 发起请求
        option = lark.RequestOption.builder().tenant_access_token(get_access_token()).build()
        response: SearchAppTableRecordResponse = client.bitable.v1.app_table_record.search(request, option)

        # 处理失败返回
        if not response.success():
            lark.logger.error(
                f"client.bitable.v1.app_table_record.search failed, code: {response.code}, msg: {response.msg}, log_id: {response.get_log_id()}, resp: \n{json.dumps(json.loads(response.raw.content), indent=4, ensure_ascii=False)}")

        current_records = response.data.items
        all_records.extend(current_records)

        # 处理业务结果
        lark.logger.info(f"已获取{len(all_records)}条记录")

        # 检查是否还有更多数据
        if not response.data.has_more:
            break  # 没有更多数据，退出循环

        # 更新分页标记，用于下一次请求
        page_token = response.data.page_token

    lark.JSON.marshal(all_records)

    data_dict = json.loads(lark.JSON.marshal(all_records))
    data_df = pd.DataFrame(index = ['公司名称','姓名','日期','电话号码','直属上级'],data =
    [[j['公司名称'][0] for j in [i['fields'] for i in data_dict]],[j['姓名'][0]['text'] for j in [i['fields'] for i in data_dict]],
     [j['日期'] for j in [i['fields'] for i in data_dict]],[j['电话号码'] for j in [i['fields'] for i in data_dict]],
     [j['直属上级'] for j in [i['fields'] for i in data_dict]]]).T

    es = 0
    len1 = len(data_df)
    data_df = data_df.drop_duplicates().reset_index(drop=True)
    len2 = len(data_df)
    if len1 != len2:
        es = 1

    count_all = {}
    for i in data_df.groupby('直属上级'):
        count_all[i[0]] = len(i[1])

    folder = r'D:/JupyterProject/data/'
    df = pd.read_excel(folder + r'Workbook.xlsx')

    df['key'] = [str(i)+str(j)[-4:] for (i,j) in zip(df['请输入姓名'],df['请输入手机号'])]
    data_df['key'] = [str(i)+str(j)[-4:] for (i,j) in zip(data_df['姓名'],data_df['电话号码'])]

    df = is_not_left(df).reset_index(drop=True)

    a = is_left_error(df)

    data_df['是否为新员工'] = is_new(data_df,df)

    count_new = {}
    for i in data_df.groupby('直属上级'):
        count_new[i[0]] = sum(i[1]['是否为新员工'])

    today = datetime.datetime.today()

    work_place_list = ['XX省XX市','YY省YY市']

    final_text = f"{today.year}年{today.month}月{today.day}日\\n今日各部门临时工打卡出勤统计播报（以临时工填写日期为准）：\\n"
    if es == 1:
        final_text += f"（今天有员工重复打卡，已去重）\\n"
    for i in count_all:
        text = f"{location_dict[i]}（直属领导：{at_someone(i,open_id_dict)}）出勤总人数{count_all[i]}人，其中新员工数{count_new[i]}人；\\n"
        final_text += text
    final_text += f"\\n腾讯文档-离职状态异常情况：\\n"
    if a:
        for err_a in a:
            text = f"离职状态异常警告，所记录人员姓名为{str(df.loc[err_a,'请输入姓名'])}，直属领导为@{at_someone(str(df.loc[err_a,'管理人员编辑直属领导']),open_id_dict)}，离职时间为{str(df.loc[err_a,'管理人员确认离职时间'])}，合同签署情况为{str(df.loc[err_a,'合同签署情况'])};\\n"
            final_text += text
    else:
        text = f"无。\\n"
        final_text += text

    check_list = ['必填项1','必填项2']
    special_check_list = ['如不是本人银行卡，请补充信息','其它辅助列1','其它辅助列2']

    err_b = []
    b= []
    for i in range(len(df)):
        err_b.append(0)
        for cl in check_list:
            if pd.isnull(df.loc[i,cl]):
                if type(df.loc[i,cl]) == str:
                    if df.loc[i,cl] == '#N/A':
                        err_b[-1]+=1
                        b.append(cl)
        if df.loc[i,'是否为本人银行卡'] == '否':
            if pd.isnull(df.loc[i,'如不是本人银行卡，请补充信息']):
                if df.loc[i,'如不是本人银行卡，请补充信息'] == ';;;':
                    err_b[-1]+=1
                    b.append('如不是本人银行卡，请补充信息')

    df['error_count'] = [1 if i > 0 else 0 for i in err_b]
    count_error = {}
    for i in df.groupby('管理人员编辑直属领导'):
        count_error[i[0]] = sum(i[1]['error_count'])

    count_error['？'] = 0
    for i in count_error:
        if i not in list(location_dict.keys()):
            count_error['？'] += count_error[i]

    my_dict = {k: v for k, v in count_error.items() if v > 0}

    df['location_error'] = [1 if i not in work_place_list else 0 for i in df['工作地']]
    le = sum(df['location_error'])

    final_text += f"\\n腾讯文档-城市异常情况：\\n"
    if le > 0:
        final_text += f"<at user_id=\\\"all\\\"></at>\\n"
        text = f"城市异常警告，存在{le}名工作地、城市异常的临时工，姓名为{'、'.join(list(df[df['location_error']==1]['请输入姓名']))}。\\n"
    else:
        text = f"无。\\n"
    final_text += text

    final_text += f"\\n腾讯文档-字段缺失情况：\\n"
    if sum([v for k,v in count_error.items()]) > 0:
        for i in my_dict:
            if i in list(location_dict.keys()):
                text = f"字段缺失警告，{str(location_dict[i])}（直属领导：{at_someone(i,open_id_dict)}）有{str(my_dict[i])}条临时工数据存在字段缺失问题;\\n"
                final_text += text
        final_text += f"\\n字段缺失提示：{'、'.join(list(set(b)))}。\\n"
    else:
        text = f"无。\\n"
        final_text += text

    df['未签署'] = [1 if i in ['已发送','签署中','发送失败'] else 0 for i in df['合同签署情况']]
    final_text += f"\\n腾讯文档-临时工未签署合同情况：\\n"
    ddict = {}
    if sum(df['未签署']) > 0:
        for i in df.groupby('管理人员编辑直属领导'):
            if sum(i[1]['未签署'])>0:
                ddict[i[0]] = sum(i[1]['未签署'])
                ddf = i[1][i[1]['未签署']==1]
                text = f"{location_dict[i[0]]}（直属领导：{at_someone(i[0],open_id_dict)}）有{ddict[i[0]]}个合同未签署员工，姓名为：{'、'.join(list(ddf['请输入姓名']))};\\n"
                final_text += text
    else:
        text = f"无。\\n"
        final_text += text

    final_text += f"<at user_id=\\\"all\\\"></at>"

    # 创建client
    client = lark.Client.builder() \
        .app_id("你的飞书开发者后台机器人ID") \
        .app_secret("你的飞书开发者后台机器人密码") \
        .log_level(lark.LogLevel.DEBUG) \
        .build()

    # 构造请求对象
    request: CreateMessageRequest = CreateMessageRequest.builder() \
        .receive_id_type('chat_id') \
        .request_body(CreateMessageRequestBody.builder()
            .receive_id("飞书群ID")
            .msg_type("text")
            .content("{\"text\":\""+final_text+"\"}")
            #.uuid()
            .build()) \
        .build()

    # 发起请求
    response: CreateMessageResponse = client.im.v1.message.create(request)

    # 处理失败返回
    if not response.success():
        lark.logger.error(
            f"client.im.v1.message.create failed, code: {response.code}, msg: {response.msg}, log_id: {response.get_log_id()}, resp: \n{json.dumps(json.loads(response.raw.content), indent=4, ensure_ascii=False)}")
    #    return

    # 处理业务结果
    lark.logger.info(lark.JSON.marshal(response.data, indent=4))

#job()
schedule.every().day.at("23:59").do(job)

while True:
    schedule.run_pending()
    time.sleep(60)